In [5]:
class param_adj:
    def __init__(self, V_data, t_data, I_data, data_steps, dt, m, n, h, g_Na, g_K, g_L, E_Na, E_K, E_L, init_guess):
        
        #variables from upload.py
        self.V0 = V_data[0]
        self.V_data = V_data
        self.I_data = I_data
        self.t_data = t_data
        self.data_steps = data_steps
        
        #simulation parameters
        self.dt = dt
        self.t_sim = np.arange(0, t_final, dt)
        
        #assign known variables
        
    
    # Define the HH model helper equations, note these are repeated from the stim_adj class, but for independent completeness included seperately
    def alpha_m(V):
        '''transition rate constant for m-gates (rapid response Na) shut gates opening as a function of voltage'''
        return 0.1 * (V + 40.0) / (1.0 - np.exp(-(V + 40.0) / 10.0))

    def beta_m(V):
        '''transition rate constant for m-gates (rapid response Na) open gates closing as a function of voltage '''
        return 4.0 * np.exp(-(V + 65.0) / 18.0)

    def alpha_h(V):
        '''transition rate constant for h-gates (slow response Na) shut gates opening as a function of voltage'''
        return 0.07 * np.exp(-(V + 65.0) / 20.0)

    def beta_h(V):
        '''transition rate constant for h-gates (slow response Na) open gates closing as a function of voltage '''
        return 1.0 / (1.0 + np.exp(-(V + 35.0) / 10.0))

    def alpha_n(V):
        '''transition rate constant for n-gates (slow response K) shut gates opening as a function of voltage'''
        return 0.01 * (V + 55.0) / (1.0 - np.exp(-(V + 55.0) / 10.0))

    def beta_n(V):
        '''transition rate constant for n-gates (slow response K) open gates closing as a function of voltage '''
        return 0.125 * np.exp(-(V + 65) / 80.0)
    
    def __forward(self, params, V, I, t):
        g_Na, g_K, g_L, E_Na, E_K, E_L, C_m, m, h, n = params
        dVdt = (I - g_Na * m**3 * h * (V - E_Na) - g_K * n**4 * (V - E_K) - g_L * (V - E_L)) / C_m
        dmdt = alpha_m(V) * (1 - m) - beta_m(V) * m
        dhdt = alpha_h(V) * (1 - h) - beta_h(V) * h
        dndt = alpha_n(V) * (1 - n) - beta_n(V) * n
        return dVdt, dmdt, dhdt, dndt
    
    def integrate_HH(self, params):
        g_Na, g_K, g_L, E_Na, E_K, E_L, C_m, m, h, n = params
        V_record = np.zeros_like(self.t_sim)
        V = self.V0
    
        for i in range(len(self.t)):
            V_record[i] = V
            dVdt, dmdt, dhdt, dndt = __forward(params, V, self.I_data[i], self.t_sim[i])
            V += dVdt * self.dt
            m += dmdt * self.dt
            h += dhdt * self.dt
            n += dndt * self.dt
        return V_record
        #forward model for unknown stimuli
        
    def __cost(self, params): 
        cost = 0
        
        V_record = []
        V = self.V0
    
        for i in range(len(self.t_sim)):
        
            # run forward step
            V_record.append(V)
        
            dVdt, dmdt, dhdt, dndt = __forward(params, V, self.I_data[i], self.t_sim[i])
            V += dVdt * self.dt
            m += dmdt * self.dt
            h += dhdt * self.dt
            n += dndt * self.dt

            # compute cost
            if self.t_sim[i] in data_steps:
                j = np.where(self.data_steps == self.t_sim[i])
                cost += (V_record[i] - self.V_data[j])**2
         
        cost = cost/len(self.data_steps)

            
        return cost
    
        
    